In [2]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.express as px 
import datetime as dt 
import ipywidgets as widgets
from IPython.display import display
import numpy as np

In [49]:
#load data 
dfusa = pd.read_csv('covid-19-data/us.csv')
dfstates = pd.read_csv('covid-19-data/us-states.csv')
dfcounties = pd.read_csv('covid-19-data/us-counties.csv')

In [50]:
def preprocess(data,s):
    data['date']=pd.to_datetime(data['date'])
    #s is wheter it is data for state or for country s==1: state, s==2:county, s==0: country
    #case for state
    if s==1:
        data.sort_values(['state','date'],inplace=True)
        states = data['state'].unique()
        data['daily_cases'] = np.zeros(data.shape[0])
        data['daily_deaths'] = np.zeros(data.shape[0])
        for s in states:
            data['daily_cases'][data['state']==s] =data.cases[data['state']==s].diff()
            data['daily_deaths'][data['state']==s] =data.deaths[data['state']==s].diff()
        neg_cases = data[data['daily_cases']<0].index
        data.drop(neg_cases, axis=0, inplace=True)
        neg_cases = data[data['daily_deaths']<0].index
        data.drop(neg_cases, axis=0, inplace=True) 
    #case for country 
    elif s==0:
        data['daily_cases'] = data['cases'].diff()
        data['daily_deaths'] = data['deaths'].diff()
        neg_cases = data[data['daily_cases']<0].index
        data.drop(neg_cases, axis=0, inplace=True)
        neg_cases = data[data['daily_deaths']<0].index
        data.drop(neg_cases, axis=0, inplace=True)
        
    else:
        data.sort_values(['county','date'],inplace=True)
        counties = data['county'].unique()
        data['daily_cases'] = data[['county','cases']].groupby(by=["county"]).diff()
        data['daily_deaths'] = data[['county','deaths']].groupby(by=["county"]).diff()
        neg_cases = data[data['daily_cases']<0].index
        data.drop(neg_cases, axis=0, inplace=True)
        neg_cases = data[data['daily_deaths']<0].index
        data.drop(neg_cases, axis=0, inplace=True) 

    
    return data    
    

In [51]:
dfusa = preprocess(dfusa,0)
dfstates= preprocess(dfstates,1)
dfcounties = preprocess(dfcounties,2)

/var/folders/07/cbtj5f0s2_v0hy9qt2t1zqgc0000gn/T/ipykernel_10743/1610212475.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['daily_cases'][data['state']==s] =data.cases[data['state']==s].diff()
/var/folders/07/cbtj5f0s2_v0hy9qt2t1zqgc0000gn/T/ipykernel_10743/1610212475.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['daily_deaths'][data['state']==s] =data.deaths[data['state']==s].diff()


In [52]:
dfcounties.head()

,date,county,state,fips,cases,deaths,daily_cases,daily_deaths
5000,2020-03-19,Abbeville,South Carolina,45001.0,1,0.0,NaN,NaN
5873,2020-03-20,Abbeville,South Carolina,45001.0,1,0.0,0.0,0.0
6877,2020-03-21,Abbeville,South Carolina,45001.0,1,0.0,0.0,0.0
7997,2020-03-22,Abbeville,South Carolina,45001.0,1,0.0,0.0,0.0
9208,2020-03-23,Abbeville,South Carolina,45001.0,1,0.0,0.0,0.0


In [54]:
#def to plot the data, if country=0 : plot for the whole country, country =1 plot for states if coountry =2,
#plot for county

def plot(y,data,state='Alabama',county='Autauga County',country=0):
    if country ==0 :
        title = '{} in the USA'.format(y)
        ax = px.area(data, 'date', y, title =title )
        ax.show()
    elif country ==1:
        title = '{} in {}'.format(y,state)
        ax = px.area(data[data['state']==state], 'date',y,title = title)
        ax.show()
    else:
        title = '{} in {}'.format(y,county)
        ax = px.area(data[data['county']==county], 'date',y,title = title)
        ax.show()
        
    

In [59]:
dropdown_case = widgets.Dropdown(options=['daily_cases','daily_deaths'])
dropdown_country = widgets.Dropdown(options=['country','state','county'])
dropdown_state = widgets.Dropdown(options=dfstates['state'].unique())
dropdown_county =  widgets.Dropdown(options=dfcounties['county'].unique())
input_widgets = widgets.HBox([dropdown_case, dropdown_country])
output = widgets.Output()
def com_filter(case, country, state,county):
    output.clear_output()
    if country == 'country':
        with output:
            plot(case,dfusa,country=0)
    elif country =='state':
        with output:
            display(dropdown_state)
            plot(case,dfstates,state=state, country=1)
    else:
        with output:
            display(dropdown_county)
            plot(case,dfcounties,county=county, country=2)

        
        
def dropdown_case_eventhandler(change):
    com_filter(change.new, dropdown_country.value,dropdown_state.value,dropdown_county.value)

def dropdown_country_eventhandler(change):
    com_filter(dropdown_case.value,change.new,dropdown_state.value,dropdown_county.value)    
    
def dropdown_state_eventhandler(change):
    com_filter(dropdown_case.value,dropdown_country.value,change.new,dropdown_county.value)
def dropdown_county_eventhandler(change):
    com_filter(dropdown_case.value,dropdown_country.value,dropdown_state.value,change.new)
    
dropdown_case.observe(dropdown_case_eventhandler, names='value')
dropdown_country.observe(dropdown_country_eventhandler, names='value')
dropdown_state.observe(dropdown_state_eventhandler, names='value')
dropdown_county.observe(dropdown_county_eventhandler, names='value')
display(input_widgets)


display(output)

Output()

Case fatality ratio (CFR) is the proportion of individuals diagnosed with a disease who die from that disease and is therefore a measure of severity among detected cases

In [84]:
#vaccination started mid december 2020 
#let's take 15 june 2020 
june15 = dfusa[dfusa['date']==dt.datetime.strptime('2020-06-15',"%Y-%m-%d")]
#December 15 2021
dec15 = dfusa[dfusa['date']==dt.datetime.strptime('2021-12-15',"%Y-%m-%d")]
CFRjune15 = float((june15['deaths']/june15['cases']))
CFRdec15 = float((dec15['deaths']/dec15['cases']))
print(CFRdec15, CFRjune15)

0.015918426836646907 0.05464940321851015


In [18]:
##anova for prisons 
pri = pd.read_csv('covid-19-data/prisons/systems.csv')

In [45]:

pri.drop([50,51,52,53],inplace=True)


KeyError: '[50 51 52 53] not found in axis'

In [54]:
pri.head()

,system,inmate_tests,total_inmate_cases,total_inmate_deaths,latest_inmate_population,max_inmate_population_2020,total_officer_cases,total_officer_deaths,total_cases,total_deaths
0,Alabama,15505.0,1601,64,19144.0,21900.0,1019.0,3.0,2620.0,67.0
1,Alaska,27756.0,2428,5,5011.0,5143.0,85.0,0.0,2513.0,5.0
2,Arizona,43652.0,12229,44,38248.0,41674.0,2739.0,0.0,14968.0,44.0
3,Arkansas,0.0,11378,52,14729.0,15886.0,424.0,5.0,11802.0,57.0
4,California,121207.0,49511,217,110471.0,117767.0,16090.0,26.0,65601.0,243.0


In [47]:
pri.describe()

,inmate_tests,total_inmate_cases,total_inmate_deaths,latest_inmate_population,max_inmate_population_2020,total_officer_cases,total_officer_deaths,total_cases,total_deaths
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,76442.720000,6902.400000,45.900000,20678.420000,23632.580000,1956.840000,3.480000,8859.240000,49.380000
std,130952.898986,8859.677962,54.334283,25759.930956,29959.091043,2892.597113,7.220266,11583.795269,59.809524
min,0.000000,169.000000,0.000000,0.000000,1397.000000,45.000000,0.000000,214.000000,0.000000
25%,12442.250000,2103.750000,9.750000,5466.250000,6053.750000,405.250000,0.000000,2536.750000,11.250000
50%,29985.000000,4099.000000,31.000000,13731.000000,15323.500000,1018.500000,1.500000,5573.500000,32.000000
75%,73595.000000,8483.750000,52.750000,24962.250000,30990.750000,2158.250000,4.000000,11288.250000,55.500000
max,673822.000000,49511.000000,220.000000,127814.000000,162000.000000,16090.000000,44.000000,65601.000000,250.000000


In [48]:
pri['total_cases'] = pri['total_inmate_cases']+pri['total_officer_cases']
pri['total_deaths'] = pri['total_inmate_deaths']+pri['total_officer_deaths']


In [55]:
#case fatality ratio 
pri['case_fat_r_in'] =(pri['total_inmate_deaths']/pri['total_inmate_cases'])*100
pri['case_fat_r_of'] =(pri['total_officer_deaths']/pri['total_officer_cases'])*100

In [56]:
pri.head()

,system,inmate_tests,total_inmate_cases,total_inmate_deaths,latest_inmate_population,max_inmate_population_2020,total_officer_cases,total_officer_deaths,total_cases,total_deaths,case_fat_r_in,case_fat_r_of
0,Alabama,15505.0,1601,64,19144.0,21900.0,1019.0,3.0,2620.0,67.0,3.997502,0.294406
1,Alaska,27756.0,2428,5,5011.0,5143.0,85.0,0.0,2513.0,5.0,0.205931,0.000000
2,Arizona,43652.0,12229,44,38248.0,41674.0,2739.0,0.0,14968.0,44.0,0.359800,0.000000
3,Arkansas,0.0,11378,52,14729.0,15886.0,424.0,5.0,11802.0,57.0,0.457022,1.179245
4,California,121207.0,49511,217,110471.0,117767.0,16090.0,26.0,65601.0,243.0,0.438286,0.161591
